In [1]:
import sqlite3
import os
import random

In [2]:
cx = sqlite3.connect('my_metrics.db')
cur = cx.cursor()

In [6]:
N = 20
para_iter = 4
batch_words = 1
table_name_prefix = 'metrics_N_%d__iter_%d__batch_words_%d'

table_name = table_name_prefix % (N, para_iter, batch_words)

In [7]:
cur.execute('select * from %s where f1 > 0.14301645' % table_name)

In [8]:
cur.fetchall()

[(37,
  460,
  1,
  1,
  0.15085714285714286,
  0.1437125748502994,
  0.14719821577920267,
  u'2017-02-18 20:47:17'),
 (71,
  360,
  1,
  1,
  0.148,
  0.14099074578116494,
  0.14441037078338445,
  u'2017-02-18 20:50:22'),
 (81,
  460,
  1,
  1,
  0.148,
  0.14099074578116494,
  0.14441037078338445,
  u'2017-02-18 20:50:41'),
 (110,
  340,
  2,
  1,
  0.14742857142857144,
  0.14044637996733805,
  0.1438528017842208,
  u'2017-02-18 20:51:31'),
 (119,
  430,
  2,
  1,
  0.14685714285714285,
  0.13990201415351117,
  0.14329523278505715,
  u'2017-02-18 20:51:49'),
 (433,
  290,
  5,
  2,
  0.14714285714285713,
  0.1401741970604246,
  0.14357401728463898,
  u'2017-02-18 21:01:14'),
 (457,
  120,
  1,
  1,
  0.14742857142857144,
  0.14044637996733805,
  0.1438528017842208,
  u'2017-02-18 21:02:02'),
 (488,
  430,
  1,
  1,
  0.14685714285714285,
  0.13990201415351117,
  0.14329523278505715,
  u'2017-02-18 21:02:58'),
 (492,
  470,
  1,
  1,
  0.14714285714285713,
  0.1401741970604246,
  0.14

In [18]:

def convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_LabeledSentence(data):
    return [gensim.models.doc2vec.LabeledSentence(map(lambda y: y[0], data[x]), [x]) for x in data]

def convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_list(data):
    return [map(lambda y: y[0], data[x]) for x in data]


In [9]:

def extract_data_from_file_and_generate_train_and_test(filename, M, k, seed, delimiter, num_of_parts_of_test_data=4, sort_by_time=False):
    test = None
    train = None
    data = {}
    random.seed(seed)

    with open(filename , 'r') as f:
        first_line = f.readline()
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            #userId = int(userId)
            #movieId = int(movieId)
            rating = float(rating)
            timestamp = int(timestamp)

            if userId not in data:
                data[userId] = []
            data[userId].append((movieId, rating, timestamp))

    test = {}
    train = {}
    for userId in data:
        total_len = len(data[userId])
        if k == random.randint(0, M):
            test[userId] = data[userId]
        else:
            train[userId] = data[userId]
    userId = None

    for userId in test:
        test[userId].sort(key=lambda x: x[2])
    # sort by time: PART 1<begin>
    if sort_by_time:
        print 'sort by time PART 1'
        for userId in train:
            train[userId].sort(key=lambda x: x[2])

    #print train[train.keys()[0]]
    #print test[test.keys()[0]]
    #raw_input()
    # sort by time: PART 1 <end>

    ### split test data further
    test_real = {}
#    if not sort_by_time:
#        for k_user in test:
#            test_real[k_user] = [[], []]
#            for m, r, t in test[k_user]:
#                # every record of test dataset is supposed to be splitted into 2 parts: input part and fact/answer part
#                # How to specify the relative of these parts? 
#                #  Assign num_of_parts_of_test_data an appropriate value: each record of test dataset is supposed
#                # to constitute num_of_parts_of_test_data parts, and one of them would serve as the fact/answer part.
#                if 0 == random.randint(0, num_of_parts_of_test_data):
#                    test_real[k_user][1].append((m, r, t)) # the fact/answer part in one record of test dataset
#                else:
#                    test_real[k_user][0].append((m, r, t)) # the input part in one record of test dataset
#    else: 
#        # sort by time: PART 2 <start>
#        print 'sort by time PART 2'
    for k_user in test:
        #print 'len(test[k_user]):', len(test[k_user])
        #print 'num_of_parts_of_test_data:', num_of_parts_of_test_data
        #raw_input()
        #print (len(test[k_user]) * (1.0 / num_of_parts_of_test_data))
        #print (int)(len(test[k_user]) * (1.0 / num_of_parts_of_test_data))
        split_point_index = -1 * ((int)(len(test[k_user]) * (1.0 / (num_of_parts_of_test_data + 1))))
        #print 'split_point_index:', split_point_index
        test_real[k_user] = [test[k_user][:split_point_index], test[k_user][split_point_index:]]
        #raw_input()

        # sort by time: PART 2 <end>

    #print test_real[test_real.keys()[0]]
    print 'sort_by_time:', sort_by_time

    #raw_input('pause')

    return train, test_real


In [14]:
data_filename, delimiter = os.path.sep.join(['ml-100k', 'u.data']), '\t'

seed = 2 
K = 10
train, test = extract_data_from_file_and_generate_train_and_test(data_filename, 4, 0, seed, delimiter)


sort_by_time: False


In [15]:
len(train)

768

In [16]:
train.keys()

['199',
 '344',
 '345',
 '340',
 '341',
 '342',
 '343',
 '810',
 '811',
 '812',
 '348',
 '349',
 '816',
 '817',
 '719',
 '718',
 '717',
 '714',
 '713',
 '712',
 '711',
 '710',
 '421',
 '129',
 '420',
 '423',
 '422',
 '425',
 '424',
 '427',
 '298',
 '299',
 '296',
 '297',
 '294',
 '295',
 '292',
 '290',
 '291',
 '591',
 '590',
 '593',
 '592',
 '595',
 '198',
 '597',
 '596',
 '195',
 '194',
 '197',
 '196',
 '191',
 '190',
 '193',
 '192',
 '270',
 '272',
 '273',
 '274',
 '275',
 '276',
 '277',
 '278',
 '279',
 '569',
 '497',
 '524',
 '525',
 '526',
 '527',
 '520',
 '521',
 '523',
 '599',
 '528',
 '449',
 '443',
 '442',
 '441',
 '440',
 '446',
 '445',
 '444',
 '108',
 '109',
 '102',
 '103',
 '101',
 '107',
 '105',
 '903',
 '900',
 '38',
 '906',
 '905',
 '33',
 '31',
 '909',
 '36',
 '35',
 '34',
 '640',
 '642',
 '645',
 '644',
 '438',
 '439',
 '436',
 '437',
 '434',
 '435',
 '432',
 '433',
 '431',
 '339',
 '335',
 '334',
 '337',
 '331',
 '330',
 '333',
 '332',
 '745',
 '855',
 '851',
 '852'

In [17]:
[len(train[k]) for k in train]

[40,
 190,
 232,
 44,
 21,
 201,
 237,
 26,
 21,
 20,
 58,
 41,
 25,
 36,
 67,
 39,
 93,
 44,
 30,
 162,
 221,
 86,
 62,
 30,
 36,
 64,
 97,
 204,
 44,
 31,
 127,
 280,
 147,
 192,
 150,
 196,
 124,
 150,
 296,
 84,
 48,
 156,
 360,
 96,
 181,
 41,
 20,
 96,
 305,
 118,
 38,
 27,
 59,
 121,
 35,
 138,
 51,
 22,
 71,
 95,
 518,
 54,
 23,
 434,
 70,
 279,
 306,
 54,
 68,
 137,
 23,
 150,
 100,
 47,
 53,
 75,
 24,
 143,
 20,
 51,
 36,
 135,
 24,
 33,
 234,
 216,
 29,
 67,
 22,
 23,
 136,
 45,
 121,
 41,
 40,
 24,
 36,
 26,
 20,
 25,
 20,
 109,
 318,
 122,
 42,
 33,
 31,
 145,
 263,
 44,
 379,
 62,
 41,
 21,
 254,
 22,
 333,
 34,
 70,
 147,
 26,
 183,
 60,
 23,
 209,
 49,
 28,
 21,
 39,
 108,
 211,
 136,
 27,
 98,
 300,
 20,
 388,
 63,
 35,
 183,
 26,
 35,
 76,
 20,
 67,
 293,
 52,
 114,
 53,
 44,
 334,
 40,
 100,
 29,
 159,
 124,
 48,
 133,
 21,
 93,
 110,
 272,
 39,
 20,
 216,
 316,
 33,
 20,
 107,
 27,
 28,
 75,
 42,
 108,
 65,
 36,
 79,
 107,
 22,
 36,
 24,
 71,
 22,
 38,
 75,
 45,
 26

In [19]:

list_of_list = convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_list(train)

In [35]:
print 'num of users:', len(list_of_list)

num of users: 768


In [32]:
print list_of_list[0]
print list_of_list[1]

print set( list_of_list[0] ) - set( list_of_list[1] )
print set( list_of_list[1] ) - set( list_of_list[0] )

['221', '268', '258', '473', '242', '324', '322', '678', '988', '294', '111', '117', '276', '285', '405', '286', '948', '475', '687', '259', '989', '508', '1', '892', '116', '1326', '93', '100', '7', '323', '313', '539', '9', '14', '408', '813', '269', '751', '243', '1354']
['479', '306', '278', '39', '462', '215', '181', '97', '1283', '119', '619', '283', '508', '716', '175', '204', '597', '537', '196', '477', '25', '663', '451', '58', '568', '707', '106', '313', '315', '111', '268', '45', '421', '190', '87', '1007', '244', '288', '148', '5', '684', '195', '125', '12', '433', '9', '248', '228', '678', '845', '274', '471', '172', '191', '245', '286', '79', '1172', '660', '213', '284', '176', '304', '303', '511', '237', '208', '1050', '559', '183', '222', '246', '1165', '472', '896', '251', '486', '255', '535', '295', '301', '73', '357', '7', '764', '815', '972', '509', '272', '955', '385', '137', '696', '291', '405', '70', '280', '129', '459', '487', '202', '647', '926', '8', '121', '3

In [22]:
from collections import Counter

In [24]:
c = Counter()

for ll in list_of_list:
    c.update(ll)
    

In [25]:
c.most_common(10)

[('50', 475),
 ('100', 428),
 ('181', 414),
 ('258', 411),
 ('294', 392),
 ('286', 388),
 ('288', 384),
 ('1', 377),
 ('121', 348),
 ('300', 346)]

In [34]:
len(list_of_list)

768

In [26]:
print 'num of items:', len(c)

1653

In [27]:
dir(c)

['__add__',
 '__and__',
 '__class__',
 '__cmp__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__missing__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 'clear',
 'copy',
 'elements',
 'fromkeys',
 'get',
 'has_key',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'most_common',
 'pop',
 'popitem',
 'setdefault',
 'subtract',
 'update',
 'values',
 'viewitems',
 'viewkeys',
 'viewvalues']

In [33]:
vl = c.values()
vl.sort(reverse=True)
print len(vl)
print vl

1653
[475, 428, 414, 411, 392, 388, 384, 377, 348, 346, 340, 335, 322, 320, 319, 317, 308, 303, 292, 285, 285, 281, 271, 269, 267, 263, 263, 262, 258, 257, 255, 254, 251, 247, 246, 245, 243, 242, 241, 240, 239, 237, 234, 233, 231, 230, 229, 228, 228, 227, 226, 223, 222, 218, 218, 214, 214, 214, 213, 212, 210, 207, 207, 206, 206, 204, 204, 202, 201, 200, 198, 198, 196, 196, 196, 196, 195, 194, 193, 192, 191, 189, 188, 187, 187, 186, 185, 184, 183, 182, 182, 181, 181, 180, 179, 178, 178, 177, 177, 177, 176, 174, 172, 172, 172, 171, 171, 171, 169, 168, 166, 165, 164, 164, 163, 163, 163, 162, 162, 161, 161, 161, 161, 160, 160, 157, 157, 156, 153, 152, 151, 151, 150, 150, 149, 148, 148, 148, 147, 147, 147, 147, 147, 146, 146, 145, 144, 144, 144, 144, 143, 143, 143, 141, 141, 141, 140, 140, 140, 139, 138, 138, 137, 137, 137, 137, 136, 136, 136, 135, 135, 135, 135, 134, 134, 133, 133, 133, 133, 133, 132, 132, 131, 131, 130, 129, 129, 129, 129, 128, 127, 127, 126, 126, 126, 125, 125, 125, 124,

In [37]:
data_filename, delimiter = os.path.sep.join(['ml-1m', 'ratings.dat']), '::'

seed = 2 
K = 10
train, test = extract_data_from_file_and_generate_train_and_test(data_filename, 4, 0, seed, delimiter)

list_of_list = convert_level_1_dict_level_2_list_of_size_3_tuples_to_list_of_list(train)

print 'num of users:', len(list_of_list)

c = Counter()

for ll in list_of_list:
    c.update(ll)

print 'num of items:', len(c)

vl = c.values()
vl.sort(reverse=True)
print len(vl)
print vl


sort_by_time: False
num of users: 4865
num of items: 3670
3670
[2790, 2397, 2386, 2299, 2148, 2142, 2113, 2076, 2073, 2067, 2020, 2018, 2014, 1972, 1964, 1920, 1857, 1851, 1833, 1810, 1808, 1806, 1804, 1800, 1773, 1763, 1747, 1732, 1665, 1654, 1608, 1590, 1572, 1542, 1448, 1445, 1421, 1417, 1412, 1406, 1394, 1388, 1375, 1371, 1369, 1367, 1363, 1359, 1359, 1349, 1349, 1332, 1322, 1314, 1309, 1300, 1294, 1273, 1271, 1253, 1249, 1244, 1234, 1227, 1210, 1208, 1200, 1199, 1195, 1190, 1182, 1165, 1163, 1156, 1151, 1146, 1141, 1134, 1133, 1128, 1126, 1126, 1125, 1125, 1119, 1108, 1103, 1100, 1093, 1091, 1090, 1088, 1086, 1084, 1076, 1074, 1071, 1070, 1069, 1069, 1067, 1064, 1062, 1056, 1054, 1047, 1047, 1042, 1030, 1030, 1024, 1012, 1012, 1005, 1001, 996, 995, 994, 993, 992, 988, 987, 979, 973, 972, 969, 968, 968, 966, 966, 964, 964, 953, 953, 952, 942, 934, 933, 932, 924, 921, 920, 914, 910, 908, 907, 905, 905, 904, 903, 902, 901, 901, 899, 896, 895, 894, 894, 891, 889, 885, 884, 882, 879, 8